In [5]:
import keras
from   sklearn.datasets import fetch_california_housing
from   sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [6]:
X, y = fetch_california_housing(return_X_y=True)

xtrainF, xtestF, ytrainF, ytestF = train_test_split(X, y, random_state=42 ,test_size=0.25)

xtrain, xvalid , ytrain , yvalid = train_test_split(xtrainF, ytrainF, random_state=42, test_size=0.25)

In [99]:
def ConstruindoModel(n_hidden, n_neurons):
    model = keras.models.Sequential()
    model.add(
        keras.layers.InputLayer(input_shape=[8])
    )
    for camadas in range(n_hidden):
        model.add(
            keras.layers.Dense( units=n_neurons, activation='relu')
        )
    model.add(keras.layers.Dense(1))

    return model

In [120]:
from scikeras.wrappers import KerasRegressor

kerasReg = KerasRegressor(
    model    = ConstruindoModel,
    n_hidden  = 2,
    n_neurons = 30,
    optimizer = 'adam',
    loss      = 'mean_squared_error',
    verbose   =0
)

In [121]:
params = dict(
    n_hidden=[4,5],
    n_neurons=[30,40],
    epochs   =[20,30]
)
print(params)
grid = GridSearchCV(kerasReg,param_grid=params, cv=3, scoring='neg_mean_absolute_error',verbose=0, refit=False)
grid.fit(xtrain,ytrain)

{'n_hidden': [4, 5], 'n_neurons': [30, 40], 'epochs': [20, 30]}


GridSearchCV(cv=3,
             estimator=KerasRegressor(loss='mean_squared_error', model=<function ConstruindoModel at 0x7f98ee71bc40>, n_hidden=2, n_neurons=30, optimizer='adam', verbose=0),
             param_grid={'epochs': [20, 30], 'n_hidden': [4, 5],
                         'n_neurons': [30, 40]},
             refit=False, scoring='neg_mean_absolute_error')

In [123]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Carregando os dados
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Definindo a função para construir o modelo
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    # Adicionando camadas densas
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    # Compilando o modelo
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Instanciando o tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='mnist_tuning')

# Procurando os melhores hiperparâmetros
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Resumo dos resultados
tuner.results_summary()

# Obtendo o melhor modelo
best_model = tuner.get_best_models(num_models=1)[0]


Trial 5 Complete [00h 05m 42s]
val_accuracy: 0.9610000054041544

Best val_accuracy So Far: 0.9724666674931844
Total elapsed time: 00h 20m 52s
Results summary
Results in my_dir/mnist_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 7
units_0: 416
units_1: 224
learning_rate: 0.0001
units_2: 192
units_3: 288
units_4: 32
units_5: 32
units_6: 32
Score: 0.9724666674931844

Trial 3 summary
Hyperparameters:
num_layers: 2
units_0: 288
units_1: 64
learning_rate: 0.0001
units_2: 96
units_3: 512
units_4: 224
units_5: 64
units_6: 448
units_7: 384
units_8: 512
units_9: 128
units_10: 224
units_11: 96
units_12: 64
units_13: 96
units_14: 384
Score: 0.9694666663805643

Trial 2 summary
Hyperparameters:
num_layers: 15
units_0: 128
units_1: 288
learning_rate: 0.001
units_2: 32
units_3: 384
units_4: 480
units_5: 384
units_6: 64
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
Score:

In [126]:
best_model